In [3]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


In [4]:
from unsloth import FastLanguageModel
import torch

# Define model parameters
max_seq_length = 2048  # Adjust according to your dataset
dtype = None  # Auto-detect dtype
load_in_4bit = True  # Use 4-bit quantization to save VRAM

# Load Llama 3.2 3B Instruct model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # Use 16 for LoRA rank (adjust as needed)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,  # No dropout to optimize
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False
)


Unsloth 2025.3.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
import pdfplumber

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Example usage
pdf_path = "path_to_your_pdf_file.pdf"
extracted_text = extract_text_from_pdf(pdf_path)


In [ ]:
def preprocess_text(text):
    # Example: Remove unwanted characters and normalize spaces
    cleaned_text = text.replace("\n", " ").strip()
    # Additional preprocessing steps can go here
    return cleaned_text

preprocessed_text = preprocess_text(extracted_text)


In [ ]:
from datasets import load_dataset

# Assuming you have your own dataset stored as a list of examples:
dataset = load_dataset('json', data_files='your_dataset_file.json', split='train')

def format_dataset_for_training(examples):
    texts = examples["extracted_texts"]  # Assuming your data has "extracted_texts" as the key
    formatted_texts = [tokenizer.apply_chat_template(text, tokenize=False, add_generation_prompt=False) for text in texts]
    return {"text": formatted_texts}

dataset = dataset.map(format_dataset_for_training, batched=True)


In [ ]:
def check_response_completeness(response):
    # Example: Check if the response ends with a proper punctuation mark or full sentence
    if response.strip()[-1] not in [".", "!", "?"]:
        return False  # If incomplete, return False
    return True

# After generating the response from the model
generated_response = tokenizer.batch_decode(outputs)[0]

if not check_response_completeness(generated_response):
    print("Incomplete response. Please ask again.")
else:
    print("Complete response:", generated_response)


In [ ]:
from datasets import load_dataset

# Assuming your dataset is in ShareGPT format or needs to be formatted to this style
dataset = load_dataset("path_to_your_dataset", split="train")

from unsloth.chat_templates import standardize_sharegpt

# Standardize dataset into the appropriate format
dataset = standardize_sharegpt(dataset)

# Apply any additional formatting if needed
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

In [ ]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

dataset[5]["conversations"]

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])


In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=60,  # Set a smaller number of steps for faster results
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs"
    ),
)
trainer.train()


In [ ]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)


In [ ]:
# Prepare the model for inference
FastLanguageModel.for_inference(model)

messages = [
    {"role": "user", "content": "What is the capital of Algeria?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

outputs = model.generate(input_ids=inputs, max_new_tokens=64, use_cache=True, temperature=1.5, min_p=0.1)
tokenizer.batch_decode(outputs)


In [ ]:
model.save_pretrained("lora_model")  # Save locally
tokenizer.save_pretrained("lora_model")

# Optionally push to the Hugging Face hub
# model.push_to_hub("your_name/lora_model", token="...")
# tokenizer.push_to_hub("your_name/lora_model", token="...")


In [ ]:
model.save_pretrained_gguf("model_3B", tokenizer, quantization_method="f16")
